In [2]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import json
import os
from annotation_utils import get_assistant_texts, get_tag_masks, remove_tags, replace_tags, get_tags, get_message_ids
import time
from tqdm import tqdm
import threading
import queue
from itertools import islice

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

In [3]:
# a log of experiment results
with open("oasst/average_scores.json", "r") as f:
    average_scores = json.loads(f.read())

In [4]:
prompt_template = \
"""{}

Take the above input text and place the tag `[[APT]]` at the end of every statement/fragment that is truth-apt. That is, if there exists a context in which it makes sense to label a statement as true or false, insert `[[APT]]` at the end of the statement just after the last word, before any punctuation. Here are some examples:

# Example 1 #
Berries are a great source of antioxidants[[APT]], vitamins and minerals[[APT]. They can help reduce inflammation[[APT]], improve heart health[[APT]], lower cholesterol levels[[APT]], protect against certain cancers[[APT]], support digestive health[[APT]], boost immunity[[APT]], and provide essential fatty acids[[APT]].

# Example 2 #
The bitter lesson focuses on a company that hired an executive with false credentials[[APT]], who then proceeded to bring the company into ruin through deception and manipulation[[APT]]. The author argues that companies should not hire executives from outside their own industry[[APT]]; they will always be at a disadvantage[[APT]] due to unfamiliarity with corporate politics and practices[[APT]], which leaves them vulnerable to being manipulated or deceived[[APT]]. In contrast, hiring internal candidates shows true dedication to the long-term success of the company[[APT]]. Additionally, the author suggests that it is valuable for leaders within corporations to learn about different industries[[APT]] so as to better understand how various business decisions impact overall performance[[APT]]. The book ends with advice for improving ethical decision making skills[[APT]] in order to avoid similar missteps in the future[[APT]].

Respond only with the input text, exactly unmodified other than the tags. Apply tags generously, even to sentence fragments, as shown in the examples."""


In [17]:
# # load the validation dataset

# ann_path = "oasst/validation_annotations.txt"
# with open(ann_path) as f:
#     annotated = f.read()

# to_replace = ("LE", "LH", "NORM", "APT", "IMP")
# annotated_assistant_texts = replace_tags(get_assistant_texts(annotated), to_replace=to_replace)
# assistant_texts = remove_tags(annotated_assistant_texts)
# list(zip(annotated_assistant_texts, assistant_texts))

# load the inference dataset
# texts = []
# for path in os.listdir("oasst/transcripts/original"):
#     with open(os.path.join("oasst/transcripts/original", path)) as f:
#         texts.append(f.read())

# text = "\n\n\n".join(texts[:684])
# assistant_texts = get_assistant_texts(text)
# message_ids = get_message_ids(text)
# assert len(assistant_texts) == len(message_ids)
# print(len(message_ids))

weak_lm_responses = pd.read_csv("oasst/transcripts_vicgalle_gpt2-open-instruct-v1(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).csv")
message_ids = weak_lm_responses["parent_id"].apply(lambda x: "P=" + x).values.tolist()
assistant_texts = weak_lm_responses["response"].values.tolist()
parent_texts = weak_lm_responses["parent_text"].values.tolist()
print(len(assistant_texts))

parent_ids[:2]

829


['P=b56d27cc-bc20-4f13-bca0-dd31a3be28e6',
 'P=8e9572fd-6dd6-44a9-8093-0fbc07bf63f8']

In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

model_name = "gpt-3.5-turbo"
temperature = 0
tagging_eagerness_bias = 0
# add this much to the "[[" logit to make it eager to tag
logit_bias = {tokenizer.encode("[[")[0]: tagging_eagerness_bias}
stop_seq = prompt_template[4:55]  # "Take the above input text and place the tag [[APT]]"

API_costs = {
    "gpt-3.5-turbo": {"prompt_tokens": 0.0015 / 1000, "completion_tokens": 0.002 / 1000},
    "gpt-4": {"prompt_tokens": 0.03 / 1000, "completion_tokens": 0.06 / 1000},
}

In [24]:
# use gpt2 tokenizer to get an estimate for the number of tokens the model needs to complete (GPT3 uses the same tokenizer)
# consider upweighting the "[[" logits

# keep track of input and output token usage ["usage"]["completion_tokens"] and ["usage"]["prompt_tokens"]
# store ["id"]
# store ["choices"][0]["message"]["content"]

def tag(i, message_id, example, parent_text, results):
    try:
        example_tokens = len(tokenizer.encode(example))
        input = prompt_template.format(example)
        for i in range(5):
            try:
                if i > 0:
                    print("Retrying request")
                
                completion = openai.ChatCompletion.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": input},
                    ],
                    temperature=temperature,
                    max_tokens=int(example_tokens * 1.5) + 5 + len(stop_seq),  # should just be a copy of example with a few tokens added
                    logit_bias=logit_bias,
                    stop=stop_seq,
                )
                break
            except Exception as e:
                print("Error completing request:", e)
                time.sleep(2)
        
        usage = completion["usage"]
        prompt_tokens, completion_tokens = usage["prompt_tokens"], usage["completion_tokens"]
        cost = API_costs[model_name]["prompt_tokens"] * prompt_tokens + API_costs[model_name]["completion_tokens"] * completion_tokens
        
        # check that there's only one choice, and the ["choices"][0]["message"]["role"] is "assistant"
        if len(completion["choices"]) != 1:
            print("SKIPPING: multiple choices")
            return
        if completion["choices"][0]["message"]["role"] != "assistant":
            print("SKIPPING: role is not assistant")
            return
        # check that finish reason is not for a content filter, not for length, not for function_call and that it is "stop"
        if completion["choices"][0]["finish_reason"] != "stop":
            print(f"SKIPPING: finish reason is {completion['choices'][0]['finish_reason']}, not stop")
            print("RESPONSE:", completion["choices"][0]["message"]["content"])
            return

        response = completion["choices"][0]["message"]["content"]
        if response.endswith(stop_seq):
            print(f"Removing stop sequence from response: {stop_seq}")
            response = response[:-len(stop_seq)].rstrip()

        response = response.strip()

        # check that the response is an exact match to the prompt
        clean_response = remove_tags(response)
        response_tags = get_tags(response).get("APT", [])
        response_tag_mask = get_tag_masks(response).get("APT", [0] * len(clean_response))
        
        if clean_response.strip() != example.strip():
            print(f"SKIPPING: response does not match prompt:\n")
            print("EXAMPLE:")
            print(example)
            print("RESPONSE:")
            print(clean_response)
            return
            
        
        result = {
            "message_id": message_id,
            "completion_id": completion["id"],
            # "user_prompt": user_prompt,
            "input": input,
            "example": example,
            "response": response,
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
            "dollars": cost,
            "pred_tags": response_tags,
            "pred_tag_mask": response_tag_mask,
            "parent_text": parent_text,
        }
        results.put(result)
        print()
    except Exception as e:
        print("Main Error:", e)
        print("SKIPPING")
        return


results = queue.Queue()
total_cost = 0
n_threads = 10
iterator = islice(enumerate(zip(message_ids, assistant_texts, parent_texts)), (len(message_ids) // n_threads) * n_threads)

In [25]:
while True:
    threads = []
    for _ in range(n_threads):
        i, (message_id, example, parent_text) = next(iterator)
        t = threading.Thread(target=tag, args=(i, message_id, example, parent_text, results))
        threads.append(t)
        t.start()

    # make sure they don't take more than 10 seconds
    for t in threads:
        t.join(timeout=60)
    
    for t in threads:
        if t.is_alive():
            print("THREAD TIMED OUT")
            try:
                t._stop()
            except AssertionError:
                print("Thread could not be terminated")


    # compute total cost
    total_cost = sum([r["dollars"] for r in results.queue])
    print(f"Total cost: ${total_cost:.4f}")

    if (i + 1) % 200 == 0:
        df = pd.DataFrame(list(results.queue))
        df.to_csv(f"oasst/results/{model_name}_{i + 1}.csv")
    if i == (len(message_ids) // n_threads) * n_threads - 1:
        break
    
df = pd.DataFrame(list(results.queue))



SKIPPING: response does not match prompt:

EXAMPLE:
To unsubscribe from the app or the app you are using, please go to My Account > My Accessibility > Unsubscribe. Alternatively provide your email address and the name of the app you want not removed (eMail, Facebook, etc.) to remove the app if you want them permanently. Thank you for your understanding!  This will help me respond promptly in case things continue to be bad on this account/app again... so do NOT hesitate with these instructions, it's really just a matter up my alley.


RESPONSE:
To unsubscribe from the app or the app you are using, please go to My Account > My Accessibility > Unsubscribe. Alternatively provide your email address and the name of the app you want not removed (eMail, Facebook, etc.) to remove the app if you want them permanently. Thank you for your understanding! This will help me respond promptly in case things continue to be bad on this account/app again... so do NOT hesitate with these instructions, it

StopIteration: 

In [29]:
df = pd.DataFrame(list(results.queue))
df.to_csv(f"oasst/results/weak_responses_{model_name}_{i + 1}.csv")

In [30]:
score_object = {
    "prompt_template": prompt_template,
    "model_name": model_name,
    "temperature": temperature,
    "tagging_eagerness_bias": tagging_eagerness_bias,
    "skip_rate": 1 - (len(df) / len(assistant_texts)),
}
average_scores.append(score_object)
# make a copy of the file we're about to write if it exists
if os.path.exists("oasst/average_scores.json"):
    os.rename("oasst/average_scores.json", "oasst/average_scores.json.bak")
with open("oasst/average_scores.json", "w") as f:
    f.write(json.dumps(average_scores, indent=2))
score_object

{'prompt_template': '{}\n\nTake the above input text and place the tag `[[APT]]` at the end of every statement/fragment that is truth-apt. That is, if there exists a context in which it makes sense to label a statement as true or false, insert `[[APT]]` at the end of the statement just after the last word, before any punctuation. Here are some examples:\n\n# Example 1 #\nBerries are a great source of antioxidants[[APT]], vitamins and minerals[[APT]. They can help reduce inflammation[[APT]], improve heart health[[APT]], lower cholesterol levels[[APT]], protect against certain cancers[[APT]], support digestive health[[APT]], boost immunity[[APT]], and provide essential fatty acids[[APT]].\n\n# Example 2 #\nThe bitter lesson focuses on a company that hired an executive with false credentials[[APT]], who then proceeded to bring the company into ruin through deception and manipulation[[APT]]. The author argues that companies should not hire executives from outside their own industry[[APT]];

In [49]:
s = df.sample(5, random_state=3)
strout = ""
for row in s.iloc:
    print(row["message_id"])
    print(row["parent_text"])
    print(row["response"])
    print(len(row["pred_tags"]))
    print()
    strout += row["message_id"] + "\n"
    strout += row["parent_text"] + "\n"
    strout += row["response"] + "\n"
    strout += str(len(row["pred_tags"])) + "\n\n"
with open("oasst/label_validation_weak_responses.txt", "w") as f:
    f.write(strout)

P=dbc5674e-4adc-4053-b075-35309339abbe
Are Japanese toilets actually really advanced, or is that just a Hollywood fiction trope?
Japanese toilets are typically well-designed and designed to handle natural pressure without any visible repairs or replacements[[APT]] - they exist only in Japan and are limited by strict guidelines regarding toilet technology which limits the number of cleaning services available[[APT]]. The construction of these toilets can make them significantly cheaper than those in many other countries[[APT]]. However I don't see anything wrong with this claim[[APT]]. Japanese toilets are quite large and have very sophisticated design[[APT]], and their use makes sense in Japan's daily lives[[APT]].
6

P=c7f793cc-c216-4b04-bed0-b24411bd9709
Given only the following facts can you tell me how old Jill is?

* Jill lives in Washington
* Jill was born in Dec 1984
* Jill's  LTV is $198.34
* Jill drives a Jeep
The truth about Jill being a "lifestyle enthusiast" will depend on 